# Importando as bibliotecas

In [82]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
from collections import Counter
import os
import sys
sys.path.insert(0, os.path.abspath('../Util'))
from dados import ProcessarDados

# Dataset

In [83]:
procData = ProcessarDados("../dataset/norm_bin_10_FEATURES_M17_CM6b_TH199.csv")

# Funções

In [84]:
# funcção que treina e testa o modelo armazenando as métricas
# retorna um dicionário cotendo os valores das métricas de cada rodada
def treinamento_teste(epocas = 10, k_folds = 5, exibir_matriz_confusao=False, exibir_metricas=False):
    
    #array para armazenar as das métricas de cada rodada
    resultados_accuracy = []
    resultados_precision = []
    resultados_recall = []
    resultados_f1 = []
    resultados_parametros = []
    
    #dicionário das métricas
    resultados_gerais = {}

    for i in range(epocas):
        # divisão os dados 
        X_train, X_test, y_train, y_test = procData.holdout(0.2)
        #print(Counter(y_test))

        # realizando o grid search para encontrar a melhor combinação entre o C, gamma e Kernel, 
        # considerando a acurácia (taxa de acerto)
        # aqui o método GridSearchCV é configurado para subdividir os dados de treino em k_folds
        
        
        clf = MLPClassifier(random_state = None)
        grid_mlp = GridSearchCV(clf, param_grid, cv=k_folds, scoring='accuracy', verbose=0, n_jobs=-1)
        grid_mlp.fit(X_train, y_train)

        # Treinando do modelo com os melhores parametros encontrados
        activation_best = grid_mlp.best_estimator_.activation
        alpha_best = grid_mlp.best_estimator_.alpha
        solver_best = grid_mlp.best_estimator_.solver
        hidden_layer_size_best = grid_mlp.best_estimator_.hidden_layer_sizes

        MLP = MLPClassifier(random_state = None, activation = activation_best, alpha = alpha_best, solver = solver_best, hidden_layer_sizes = hidden_layer_size_best)
        MLP.fit(X_train, y_train)

        #testando o modelo
        y_pred = MLP.predict(X_test)
        if exibir_matriz_confusao:
            print(confusion_matrix(y_test, y_pred))

        # calculado as metricas
        accuracy = metrics.accuracy_score(y_test, y_pred)
        precision = metrics.precision_score(y_test, y_pred)
        recall = metrics.recall_score(y_test, y_pred)
        f1_score = 2 * (precision * recall) / (precision + recall)

        # armazenando as métricas
        resultados_accuracy.append(accuracy)
        resultados_precision.append(precision)
        resultados_recall.append(recall)
        resultados_f1.append(f1_score)

        best_parametros = "Activation: "+ str(activation_best)+ " | Alpha: "+ str(alpha_best)+ " | Solver: "+ str(solver_best) + " | hidden_layer_sizes: "+ str(hidden_layer_size_best);
        resultados_parametros.append(best_parametros)


        if exibir_metricas:
            print("Rodada: #",i)
            print(best_parametros)
            print("Accuracy:",accuracy)
            print("Precision:",precision)
            print("Recall:",recall)
            print("F1-Score:",f1_score)
            print("\n")

            
    resultados_gerais['accuracy'] = resultados_accuracy
    resultados_gerais['precision'] = resultados_precision
    resultados_gerais['recall'] = resultados_recall
    resultados_gerais['f1'] = resultados_f1
    
    return resultados_gerais

In [85]:
def tabelaMetricas(nome_modelo, dict_metricas, rodadas=False, salvarResultados=True):
        
    print ("============================================== "+nome_modelo+" =================================================")
    print ("=================================== TABELA DE MÉTRICAS DO MODELO ===================================")
    
    if(rodadas==False):
        print ("\t Accuracy \t|\t Precision \t|\t Recall \t|\t F1-Score")
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['accuracy'], axis=0), np.std(dict_metricas['accuracy'], axis=0)),end='       ')
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['precision'], axis=0), np.std(dict_metricas['precision'], axis=0)),end='    ')
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['recall'], axis=0), np.std(dict_metricas['recall'], axis=0)),end='       ')
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['f1'], axis=0), np.std(dict_metricas['f1'], axis=0)))
        print ("====================================================================================================")
        
    if(salvarResultados):
        # save to npy file
        np.save('../resultados/resultados_'+nome_modelo+'.npy', dict_metricas)

# Definindo os parâmetros

In [86]:
activations = ['relu']
alphas = 10.0 ** -np.arange(1, 4)
solvers = ['lbfgs']
hidden_layer_sizes = [100, 150, 200]
param_grid = {'hidden_layer_sizes': hidden_layer_sizes, 'activation' : activations, 'alpha': alphas, 'solver': solvers}

epocas = 50
k_folds = 5
exibir_matriz_confusao = True
exibir_metricas = True
salvarResultados = True
rodadas=False

# Treinando e obtendo as métricas do modelo

In [87]:
# treinando o modelo
dict_metricas = treinamento_teste(epocas, k_folds, exibir_matriz_confusao, exibir_metricas)
tabelaMetricas('MLP',dict_metricas, rodadas, salvarResultados)

[[18  7]
 [ 3 92]]
Rodada: # 0
Activation: relu | Alpha: 0.1 | Solver: lbfgs | hidden_layer_sizes: 150
Accuracy: 0.9166666666666666
Precision: 0.9292929292929293
Recall: 0.968421052631579
F1-Score: 0.9484536082474226


[[20  8]
 [ 5 87]]
Rodada: # 1
Activation: relu | Alpha: 0.1 | Solver: lbfgs | hidden_layer_sizes: 150
Accuracy: 0.8916666666666667
Precision: 0.9157894736842105
Recall: 0.9456521739130435
F1-Score: 0.9304812834224598


[[28  2]
 [14 76]]
Rodada: # 2
Activation: relu | Alpha: 0.001 | Solver: lbfgs | hidden_layer_sizes: 150
Accuracy: 0.8666666666666667
Precision: 0.9743589743589743
Recall: 0.8444444444444444
F1-Score: 0.9047619047619048


[[27  5]
 [ 2 86]]
Rodada: # 3
Activation: relu | Alpha: 0.1 | Solver: lbfgs | hidden_layer_sizes: 200
Accuracy: 0.9416666666666667
Precision: 0.945054945054945
Recall: 0.9772727272727273
F1-Score: 0.9608938547486033


[[22  6]
 [ 5 87]]
Rodada: # 4
Activation: relu | Alpha: 0.1 | Solver: lbfgs | hidden_layer_sizes: 200
Accuracy: 0.90833

[[20  8]
 [ 4 88]]
Rodada: # 38
Activation: relu | Alpha: 0.1 | Solver: lbfgs | hidden_layer_sizes: 100
Accuracy: 0.9
Precision: 0.9166666666666666
Recall: 0.9565217391304348
F1-Score: 0.9361702127659574


[[27  6]
 [ 9 78]]
Rodada: # 39
Activation: relu | Alpha: 0.1 | Solver: lbfgs | hidden_layer_sizes: 200
Accuracy: 0.875
Precision: 0.9285714285714286
Recall: 0.896551724137931
F1-Score: 0.912280701754386


[[25  4]
 [ 1 90]]
Rodada: # 40
Activation: relu | Alpha: 0.1 | Solver: lbfgs | hidden_layer_sizes: 100
Accuracy: 0.9583333333333334
Precision: 0.9574468085106383
Recall: 0.989010989010989
F1-Score: 0.972972972972973


[[25  7]
 [ 2 86]]
Rodada: # 41
Activation: relu | Alpha: 0.1 | Solver: lbfgs | hidden_layer_sizes: 200
Accuracy: 0.925
Precision: 0.9247311827956989
Recall: 0.9772727272727273
F1-Score: 0.9502762430939227


[[24  4]
 [ 7 85]]
Rodada: # 42
Activation: relu | Alpha: 0.1 | Solver: lbfgs | hidden_layer_sizes: 100
Accuracy: 0.9083333333333333
Precision: 0.955056179775280